In [ ]:
# This notebook is used to analyse the chosen combinis in detail.
# It creates a csv file with all the evaluation restults for all 60 models for each of the tests.
# All the tests with length, noise, starting state etc. are being analysed here.
# Lots of statistics and plots are produced.

In [ ]:
# Loads or creates csv file with all evaluations to safe time for next execution
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import iqr

from src.main.rl.evaluation.plots.phase3_plots import (
    plot_actions_taken,
    plot_observations,
)
from src.main.rl.evaluation.phase3_evaluation import create_evaluation_df_phase3
from src.main.rl.utils.constants import color_mapping

paths = [
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_SAC_training_04_06",
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption3Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
]
path_to_save = (
    "src/main/rl/evaluation/output/phase3_evaluation_results_with_new_combi1.csv"
)
os.makedirs("src/main/rl/evaluation/output/",exist_ok=True)
os.makedirs("src/main/rl/evaluation/plot_results/",exist_ok=True)

pd.options.display.max_colwidth = 500
df = pd.DataFrame()
try:
    df = pd.read_csv(path_to_save)
except:
    pass
if df.empty:
    create_evaluation_df_phase3(path_to_save, paths)
    df = pd.read_csv(path_to_save)
len(df)

In [ ]:
# Statistics per combination
df_statistics_per_combination = (
    df.drop(columns=["full_path"])
    .groupby(
        [
            "combination",
            "scenario",
            "alg",
            "action_wrapper",
            "obs_wrapper",
            "automation_wrapper",
        ],
        dropna=False,
    )
    .agg(["mean", "max", "min", "std", iqr])
)
# Necessary to set index as alphabetical is confusing for the thesis
df_statistics_per_combination["index"] = [0, 1, 3, 2, 5, 4]
df_statistics_per_combination = (
    df_statistics_per_combination.reset_index()
    .set_index(["index", "combination"])
    .sort_index()
)
df_statistics_per_combination.columns = [
    "_".join(a) for a in df_statistics_per_combination.columns.to_flat_index()
]
save_df_per_combination = df_statistics_per_combination.copy()
df_statistics_per_combination

In [ ]:
# Criticality Score per combination
cols = [item for item in df_statistics_per_combination.columns if "criti" in item]
df_statistics_per_combination[cols].round(2)

In [ ]:
# SOP Analysis
from src.main.rl.evaluation.eval import evaluate_sop

(
    cum_reward_sop,
    criticality_score_sop,
    total_timesteps_sop,
    actions_taken_sop,
    obs_taken_sop,
    info_sop,
) = evaluate_sop()

In [ ]:
# Getting actions and observations from the chosen combination. In this case the SOP will be compared to combination 1.
from src.main.rl.evaluation.plots.phase3_plots import (
    prepare_one_combination_actions_and_obs_for_analysis,
)
from src.main.rl.utils.constants import color_mapping
from src.main.rl.utils.constants import (
    scaling_factors_scenario_1,
    action_dimensions_german,
    obs_scaling_factors,
)
import matplotlib.pyplot as plt

path = "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_SAC_training_04_06"
(
    actions_prepared,
    obs_prepared,
    list_of_all_actions_taken,
    list_of_all_obs_taken,
) = prepare_one_combination_actions_and_obs_for_analysis(path, obs_dimensions=6)

for idx, item in enumerate(actions_prepared):
    # actions are scaled in the models, so we need to rescale
    item = item.applymap(
        lambda x: int(round((x + 1) * (scaling_factors_scenario_1[idx] / 2)))
    )
    # get min, mean, max for plotting
    actions_prepared[idx] = item.agg(["mean"], axis=1)

for idx, item in enumerate(obs_prepared):
    item = item.fillna(0)
    # rescaling obs
    item = item.applymap(
        lambda x: int(round((x + 1) * (obs_scaling_factors[6][idx] / 2)))
    )
    obs_prepared[idx] = item.agg(["mean"], axis=1)

In [ ]:
# Actions from SOP are printed in blue and the average action taken from combi 1 is printed in grey
y_axis_scale_actions = [[0, 100], [0, 2200], [0, 1.1], [0, 1.1], [0, 2200]]

fig, ax = plot_actions_taken(
    actions_taken_sop, "scenario1", y_axis_scale_actions, color="standard"
)
[
    ax[idx].plot(item["mean"], color=color_mapping["grey"])
    for idx, item in enumerate(actions_prepared)
]

fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_action_sop_combi1.png",
    format="png",
    dpi=300,
)

In [ ]:
# Obs from SOP are printed in blue and the average obs taken from combi 1 is printed in grey
y_axis_scale_obs = [[0, 1000], [0, 4000], [0, 550], [0, 8000], [0, 180], [0, 32]]
fig, ax = plot_observations(obs_taken_sop, y_axis_scale_obs)
[
    ax[idx].plot(item["mean"], color=color_mapping["grey"])
    for idx, item in enumerate(obs_prepared)
]
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_obs_sop_combi1.png",
    format="png",
    dpi=300,
)

In [ ]:
# 1000 length
# Models which have not made the 1000 steps (out of 60)
df[df["episode_length_1000_timesteps"] != 1000]

In [ ]:
# Number of succesfull models doing 1000 steps per combination.
length_1000 = (
    df[["episode_length_1000_timesteps", "combination"]]
    .set_index("combination")
    .eq(1000)
    .groupby("combination")
    .sum()
    .transpose()
    .sum()
    .transpose()
    .to_latex()
)
print(length_1000)

In [ ]:
# Check how often the pumps are blown in the 1000 length test
df[
    [
        "episode_length_1000_condensator_pump_blown",
        "episode_length_1000_water_pump_blown",
    ]
].sum()

In [ ]:
# Noise
cols = [item for item in df_statistics_per_combination.columns if "DelayNoise" in item]
df_statistics_per_combination[cols].round(2)
# Print statistics for different noise options per combination (return)
print(
    df_statistics_per_combination[cols]
    .round(2)
    .droplevel(1)[
        [
            "DelayNoiseWrapperOption1_mean",
            "DelayNoiseWrapperOption1_max",
            "DelayNoiseWrapperOption1_min",
            "DelayNoiseWrapperOption1_std",
            "DelayNoiseWrapperOption1_iqr",
            "DelayNoiseWrapperOption2_mean",
            "DelayNoiseWrapperOption2_max",
            "DelayNoiseWrapperOption2_min",
            "DelayNoiseWrapperOption2_std",
            "DelayNoiseWrapperOption2_iqr",
        ]
    ]
    .transpose()
    .to_latex()
)

In [ ]:
# Box Plots for noise per option and per combiniation using timesteps
def set_box_color(bp, color, idx):
    plt.setp(bp["boxes"], color=color)
    plt.setp(bp["whiskers"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["medians"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["fliers"], color=color)


noise_df = df[
    [
        "obs_wrapper",
        "automation_wrapper",
        "scenario",
        "DelayNoiseWrapperOption2_timesteps",
        "DelayNoiseWrapperOption1_timesteps",
        "combination",
        "ObservationVariesNoiseWrapper1_timesteps",
        "ObservationVariesPositiveNoiseWrapper_timesteps",
        "ObservationVariesNegativeNoiseWrapper_timesteps",
    ]
]

order_of_col = [
    "ObservationVariesPositiveNoiseWrapper_timesteps",
    "ObservationVariesNegativeNoiseWrapper_timesteps",
    "ObservationVariesNoiseWrapper1_timesteps",
    "DelayNoiseWrapperOption1_timesteps",
    "DelayNoiseWrapperOption2_timesteps",
]
fig, ax = plt.subplots(constrained_layout=True)
fig.set_figwidth(14)

ax1 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_SAC_training_04_06'"
    )[order_of_col],
    positions=[0, 0.2, 0.4, 0.6, 0.8],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax2 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06'"
    )[order_of_col],
    positions=[1.2, 1.4, 1.6, 1.8, 2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax3 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[2.4, 2.6, 2.8, 3, 3.2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax4 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[3.6, 3.8, 4, 4.2, 4.4],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax5 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[4.8, 5, 5.2, 5.4, 5.6],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax6 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption3Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[6, 6.2, 6.4, 6.6, 6.8],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
set_box_color(ax1, color_mapping["blue"], 0)
set_box_color(ax1, color_mapping["blue"], 1)
set_box_color(ax2, color_mapping["red"], 0)
set_box_color(ax2, color_mapping["red"], 1)
set_box_color(ax3, color_mapping["grey"], 0)
set_box_color(ax3, color_mapping["grey"], 1)
set_box_color(ax4, color_mapping["yellow"], 0)
set_box_color(ax4, color_mapping["yellow"], 1)
set_box_color(ax5, color_mapping["brown"], 0)
set_box_color(ax5, color_mapping["brown"], 1)
set_box_color(ax6, color_mapping["turquoise"], 0)
set_box_color(ax6, color_mapping["turquoise"], 1)
ax.set_xlabel("Rauschoptionen")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["blue"], label="Kombination 1")
plt.plot([], c=color_mapping["red"], label="Kombination 2")
plt.plot([], c=color_mapping["grey"], label="Kombination 3")
plt.plot([], c=color_mapping["yellow"], label="Kombination 4")
plt.plot([], c=color_mapping["brown"], label="Kombination 5")
plt.plot([], c=color_mapping["turquoise"], label="Kombination 6")
plt.legend(loc=[0.2, 0.1])

plt.show()
plt.tight_layout = True

In [ ]:
# How many models have successfully completed the 250 timesteps per noise option
noise_df.eq(250).sum()

In [ ]:
# Box Plots for noise per option and per combiniation using return
def set_box_color(bp, color, idx):
    plt.setp(bp["boxes"], color=color)
    plt.setp(bp["whiskers"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["medians"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["fliers"], color=color)


noise_df = df[
    [
        "obs_wrapper",
        "automation_wrapper",
        "scenario",
        "DelayNoiseWrapperOption2",
        "DelayNoiseWrapperOption1",
        "combination",
        "ObservationVariesNoiseWrapper1",
        "ObservationVariesPositiveNoiseWrapper",
        "ObservationVariesNegativeNoiseWrapper",
    ]
]

order_of_col = [
    "ObservationVariesPositiveNoiseWrapper",
    "ObservationVariesNegativeNoiseWrapper",
    "ObservationVariesNoiseWrapper1",
    "DelayNoiseWrapperOption1",
    "DelayNoiseWrapperOption2",
]
fig, ax = plt.subplots(constrained_layout=True)
fig.set_figwidth(14)

ax1 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_SAC_training_04_06'"
    )[order_of_col],
    positions=[0, 0.2, 0.4, 0.6, 0.8],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax2 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06'"
    )[order_of_col],
    positions=[1.2, 1.4, 1.6, 1.8, 2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax3 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[2.4, 2.6, 2.8, 3, 3.2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax4 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[3.6, 3.8, 4, 4.2, 4.4],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax5 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[4.8, 5, 5.2, 5.4, 5.6],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax6 = ax.boxplot(
    noise_df.query(
        "combination=='src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption3Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[6, 6.2, 6.4, 6.6, 6.8],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
set_box_color(ax1, color_mapping["blue"], 0)
set_box_color(ax1, color_mapping["blue"], 1)
set_box_color(ax2, color_mapping["red"], 0)
set_box_color(ax2, color_mapping["red"], 1)
set_box_color(ax3, color_mapping["grey"], 0)
set_box_color(ax3, color_mapping["grey"], 1)
set_box_color(ax4, color_mapping["yellow"], 0)
set_box_color(ax4, color_mapping["yellow"], 1)
set_box_color(ax5, color_mapping["brown"], 0)
set_box_color(ax5, color_mapping["brown"], 1)
set_box_color(ax6, color_mapping["turquoise"], 0)
set_box_color(ax6, color_mapping["turquoise"], 1)
ax.set_xlabel("Rauschoptionen")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["blue"], label="Kombination 1")
plt.plot([], c=color_mapping["red"], label="Kombination 2")
plt.plot([], c=color_mapping["grey"], label="Kombination 3")
plt.plot([], c=color_mapping["yellow"], label="Kombination 4")
plt.plot([], c=color_mapping["brown"], label="Kombination 5")
plt.plot([], c=color_mapping["turquoise"], label="Kombination 6")
plt.legend(loc=[0.2, 0.1])

plt.show()
plt.tight_layout = True

In [ ]:
# Successfull number of model-noise combinations per combination (max 50 -> 5 noise options * 10 models)
# Successfull meaning here to have completed the 250 timesteps fully
noise_agg = (
    df[
        [
            "DelayNoiseWrapperOption2_timesteps",
            "DelayNoiseWrapperOption1_timesteps",
            "ObservationVariesNoiseWrapper1_timesteps",
            "ObservationVariesNoiseWrapper1_timesteps",
            "ObservationVariesNegativeNoiseWrapper_timesteps",
            "combination",
        ]
    ]
    .set_index("combination")
    .eq(250)
    .groupby("combination")
    .sum()
    .transpose()
    .sum()
    .transpose()
    .to_latex()
)
print(noise_agg)

In [ ]:
# Starting State
cols = [col for col in df.columns if "starting" in col and "criticality" not in col]
# Mean auf all strating state options (return + timesteps)
df[cols].mean()

In [ ]:
# Starting state 1 - Models that were not executing the 250 timesteps completly
df[df["create_starting_state_option1_timesteps"] != 250]

In [ ]:
# Box Plots for starting state 2 per option and per combination using timesteps
def set_box_color(bp, color, idx):
    plt.setp(bp["boxes"], color=color)
    plt.setp(bp["whiskers"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["medians"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["fliers"], color=color)


starting_state2 = df[
    [
        "combination",
        "create_starting_state_option2a_timesteps",
        "create_starting_state_option2b_timesteps",
        "create_starting_state_option2c_timesteps",
    ]
]

order_of_col = [
    "create_starting_state_option2a_timesteps",
    "create_starting_state_option2b_timesteps",
    "create_starting_state_option2c_timesteps",
]
fig, ax = plt.subplots(constrained_layout=True)
fig.set_figwidth(8)
ax1 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_SAC_training_04_06'"
    )[order_of_col],
    positions=[
        0,
        0.2,
        0.4,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax2 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06'"
    )[order_of_col],
    positions=[
        0.9,
        1.1,
        1.3,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax3 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[1.8, 2, 2.2],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax4 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[2.7, 2.9, 3.1],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax5 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[3.6, 3.8, 4],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax6 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption3Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[4.5, 4.7, 4.9],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
set_box_color(ax1, color_mapping["blue"], 0)
set_box_color(ax1, color_mapping["blue"], 1)
set_box_color(ax2, color_mapping["red"], 0)
set_box_color(ax2, color_mapping["red"], 1)
set_box_color(ax3, color_mapping["grey"], 0)
set_box_color(ax3, color_mapping["grey"], 1)
set_box_color(ax4, color_mapping["yellow"], 0)
set_box_color(ax4, color_mapping["yellow"], 1)
set_box_color(ax5, color_mapping["brown"], 0)
set_box_color(ax5, color_mapping["brown"], 1)
set_box_color(ax6, color_mapping["turquoise"], 0)
set_box_color(ax6, color_mapping["turquoise"], 1)
ax.set_xlabel("Startzustände")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["blue"], label="Kombination 1")
plt.plot([], c=color_mapping["red"], label="Kombination 2")
plt.plot([], c=color_mapping["grey"], label="Kombination 3")
plt.plot([], c=color_mapping["yellow"], label="Kombination 4")
plt.plot([], c=color_mapping["brown"], label="Kombination 5")
plt.plot([], c=color_mapping["turquoise"], label="Kombination 6")
plt.legend(loc=[0.35, 0.5])
plt.show()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_starting_state_2_by_combi.png",
    format="png",
    dpi=300,
)

In [ ]:
# How many models have successfully completed the 250 timesteps per starting state option
starting_state2.eq(250).sum()

In [ ]:
# Box Plots for starting state 2 per option and per combination using return
def set_box_color(bp, color, idx):
    plt.setp(bp["boxes"], color=color)
    plt.setp(bp["whiskers"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["medians"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["fliers"], color=color)


starting_state2 = df[
    [
        "combination",
        "create_starting_state_option2a",
        "create_starting_state_option2b",
        "create_starting_state_option2c",
    ]
]

order_of_col = [
    "create_starting_state_option2a",
    "create_starting_state_option2b",
    "create_starting_state_option2c",
]
fig, ax = plt.subplots(constrained_layout=True)
fig.set_figwidth(8)
ax1 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_SAC_training_04_06'"
    )[order_of_col],
    positions=[
        0,
        0.2,
        0.4,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax2 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06'"
    )[order_of_col],
    positions=[
        0.9,
        1.1,
        1.3,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax3 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[1.8, 2, 2.2],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax4 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[2.7, 2.9, 3.1],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax5 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[3.6, 3.8, 4],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax6 = ax.boxplot(
    starting_state2.query(
        "combination=='src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption3Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06'"
    )[order_of_col],
    positions=[4.5, 4.7, 4.9],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
set_box_color(ax1, color_mapping["blue"], 0)
set_box_color(ax1, color_mapping["blue"], 1)
set_box_color(ax2, color_mapping["red"], 0)
set_box_color(ax2, color_mapping["red"], 1)
set_box_color(ax3, color_mapping["grey"], 0)
set_box_color(ax3, color_mapping["grey"], 1)
set_box_color(ax4, color_mapping["yellow"], 0)
set_box_color(ax4, color_mapping["yellow"], 1)
set_box_color(ax5, color_mapping["brown"], 0)
set_box_color(ax5, color_mapping["brown"], 1)
set_box_color(ax6, color_mapping["turquoise"], 0)
set_box_color(ax6, color_mapping["turquoise"], 1)
ax.set_xlabel("Startzustände")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["blue"], label="Kombination 1")
plt.plot([], c=color_mapping["red"], label="Kombination 2")
plt.plot([], c=color_mapping["grey"], label="Kombination 3")
plt.plot([], c=color_mapping["yellow"], label="Kombination 4")
plt.plot([], c=color_mapping["brown"], label="Kombination 5")
plt.plot([], c=color_mapping["turquoise"], label="Kombination 6")
plt.legend(loc=[0.35, 0.5])
plt.show()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_starting_state_2_by_combi.png",
    format="png",
    dpi=300,
)

In [ ]:
# Starting state 3 per combination mean timesteps completed
starting_state3 = df[
    [
        "obs_wrapper",
        "combination",
        "automation_wrapper",
        "scenario",
        "create_starting_state_option3_timesteps",
    ]
]
starting_state3.groupby("combination").agg("mean")

In [ ]:
# Starting state 3 completed 250 timesteps (number of models out of 60)
starting_state3.eq(250).sum(axis=1).sum()

In [ ]:
# Starting state options per combination timestep statistics
df_statistics_per_combination[
    [
        "create_starting_state_option1_timesteps_mean",
        "create_starting_state_option2a_timesteps_mean",
        "create_starting_state_option2b_timesteps_mean",
        "create_starting_state_option2c_timesteps_mean",
        "create_starting_state_option3_timesteps_mean",
    ]
].agg(["mean", "max", "min", "std"], axis=1)

In [ ]:
# Starting state options per model return statistics
df[
    [
        "create_starting_state_option1",
        "create_starting_state_option2a",
        "create_starting_state_option2b",
        "create_starting_state_option2c",
        "create_starting_state_option3",
    ]
].agg(["mean", "max", "min", "std"], axis=1)

In [ ]:
# Starting state options per combination timestep statistics
print(
    df_statistics_per_combination[
        [
            "create_starting_state_option1_timesteps_mean",
            "create_starting_state_option2a_timesteps_mean",
            "create_starting_state_option2b_timesteps_mean",
            "create_starting_state_option2c_timesteps_mean",
            "create_starting_state_option3_timesteps_mean",
        ]
    ]
    .agg(["mean", "max", "min", "std"], axis=1)
    .droplevel(1)
    .round(2)
    .transpose()
    .to_latex()
)

In [ ]:
# Successfull number of model-starting-state combinations per combination (max 50 -> 5 noise options * 10 models)
# Successfull meaning here to have completed the 250 timesteps fully
starting_state_2 = (
    df[
        [
            "create_starting_state_option1_timesteps",
            "create_starting_state_option2a_timesteps",
            "create_starting_state_option2b_timesteps",
            "create_starting_state_option2c_timesteps",
            "create_starting_state_option3_timesteps",
            "combination",
        ]
    ]
    .set_index("combination")
    .eq(250)
    .groupby("combination")
    .sum()
    .transpose()
    .sum()
    .transpose()
    .to_latex()
)

print(starting_state_2)

In [ ]:
# Number of model-experiment combinations that sucessfully executed all timesteps during the specific experiment.
# In total there were 660 such combinations
(
    df[
        [
            "DelayNoiseWrapperOption2_timesteps",
            "DelayNoiseWrapperOption1_timesteps",
            "ObservationVariesNoiseWrapper1_timesteps",
            "ObservationVariesNoiseWrapper1_timesteps",
            "ObservationVariesNegativeNoiseWrapper_timesteps",
            "create_starting_state_option1_timesteps",
            "create_starting_state_option2a_timesteps",
            "create_starting_state_option2b_timesteps",
            "create_starting_state_option2c_timesteps",
            "create_starting_state_option3_timesteps",
        ]
    ]
    .eq(250)
    .sum()
    .sum()
    + df["episode_length_1000_timesteps"].eq(1000).sum()
)

In [ ]:
# Number of model-experiment combinations per combination that sucessfully executed all timesteps during the specific experiment.
# In total there were 110 such combinations per combination
# Carefull: The order of the combination might not be the same as in the thesis!


def get_counts_by_group(df, max_timesteps):
    return df.eq(max_timesteps).sum()


counts_noise = (
    df[
        [
            "combination",
            "DelayNoiseWrapperOption2_timesteps",
            "DelayNoiseWrapperOption1_timesteps",
            "ObservationVariesNoiseWrapper1_timesteps",
            "ObservationVariesNoiseWrapper1_timesteps",
            "ObservationVariesNegativeNoiseWrapper_timesteps",
        ]
    ]
    .groupby("combination")
    .apply(get_counts_by_group, (250))
).sum(axis=1)
counts_starting_state = (
    df[
        [
            "combination",
            "create_starting_state_option1_timesteps",
            "create_starting_state_option2a_timesteps",
            "create_starting_state_option2b_timesteps",
            "create_starting_state_option2c_timesteps",
            "create_starting_state_option3_timesteps",
        ]
    ]
    .groupby("combination")
    .apply(get_counts_by_group, (250))
).sum(axis=1)
counts_1000 = (
    df[["combination", "episode_length_1000_timesteps"]]
    .groupby("combination")
    .apply(get_counts_by_group, (1000))["episode_length_1000_timesteps"]
)

print("Counts over all modification experiments with noise:")
print(counts_noise)
print("Counts over all modification experiments with starting states:")
print(counts_starting_state)
print("Counts over all modification experiment with 100 length:")
print(counts_1000)

print("Counts over all modification experiments:")
print(counts_noise + counts_starting_state + counts_1000)